## Data preprocessing for the Fake News Detection

In [40]:
# imports
import nltk
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [26]:
fake=pd.read_csv('../data/fake.csv')
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [27]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [28]:
fake.duplicated().sum()

3

In [29]:
fake.drop_duplicates(inplace=True)

In [30]:
fake.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [31]:
true=pd.read_csv('../data/true.csv')
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [32]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [33]:
true.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [34]:
true.duplicated().sum()

206

In [35]:
true.drop_duplicates(inplace=True)

### Feature Engineering

In [36]:
true['label']=0
fake['label']=1

df=pd.concat([true,fake],axis=0,ignore_index=True)
df=df.sample(frac=1,random_state=42).reset_index(drop=True)
df.head()

,title,text,subject,date,label
0,"Merkel, Juncker discuss Catalan crisis: EU off...",BRUSSELS (Reuters) - German Chancellor Angela ...,worldnews,"October 7, 2017",0
1,LIBERAL HACK KATIE COURIC Says Fake News Is “T...,Katie Couric is a wolf in sheep s clothing. Da...,left-news,"Jul 22, 2017",1
2,TRUMP WAS RIGHT! Audit Reveals State Dept. Cyb...,New documents appear to show a pattern of dete...,politics,"Oct 19, 2015",1
3,Clay Aiken Says He Was A ‘F*****g Dumbass’ Fo...,Former American Idol finalist and Celebrity...,News,"August 16, 2017",1
4,U.S. militia girds for trouble as presidential...,"JACKSON, Ga. (Reuters) - Down a Georgia count...",politicsNews,"November 2, 2016",0


In [37]:
# Convert the 'date' column to datetime format

df['date'] = pd.to_datetime(df['date'],format='mixed', dayfirst=True,errors='coerce')
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month_name()

df.head()


,title,text,subject,date,label,Year,Month
0,"Merkel, Juncker discuss Catalan crisis: EU off...",BRUSSELS (Reuters) - German Chancellor Angela ...,worldnews,2017-10-07,0,2017.0,October
1,LIBERAL HACK KATIE COURIC Says Fake News Is “T...,Katie Couric is a wolf in sheep s clothing. Da...,left-news,2017-07-22,1,2017.0,July
2,TRUMP WAS RIGHT! Audit Reveals State Dept. Cyb...,New documents appear to show a pattern of dete...,politics,2015-10-19,1,2015.0,October
3,Clay Aiken Says He Was A ‘F*****g Dumbass’ Fo...,Former American Idol finalist and Celebrity...,News,2017-08-16,1,2017.0,August
4,U.S. militia girds for trouble as presidential...,"JACKSON, Ga. (Reuters) - Down a Georgia count...",politicsNews,2016-11-02,0,2016.0,November


In [38]:
df.columns = df.columns.str.replace('_', ' ').str.title()
df.head()

,Title,Text,Subject,Date,Label,Year,Month
0,"Merkel, Juncker discuss Catalan crisis: EU off...",BRUSSELS (Reuters) - German Chancellor Angela ...,worldnews,2017-10-07,0,2017.0,October
1,LIBERAL HACK KATIE COURIC Says Fake News Is “T...,Katie Couric is a wolf in sheep s clothing. Da...,left-news,2017-07-22,1,2017.0,July
2,TRUMP WAS RIGHT! Audit Reveals State Dept. Cyb...,New documents appear to show a pattern of dete...,politics,2015-10-19,1,2015.0,October
3,Clay Aiken Says He Was A ‘F*****g Dumbass’ Fo...,Former American Idol finalist and Celebrity...,News,2017-08-16,1,2017.0,August
4,U.S. militia girds for trouble as presidential...,"JACKSON, Ga. (Reuters) - Down a Georgia count...",politicsNews,2016-11-02,0,2016.0,November


In [39]:
# Cleaning text using regex

stopword = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

df['Text'] = df['Text'].apply(lambda x: clean(x))
df['Title'] = df['Title'].apply(lambda x: clean(x))


In [41]:
# Handle missing values in the 'Text' column
df['Text'] = df['Text'].fillna('')

In [42]:
df.head()

,Title,Text,Subject,Date,Label,Year,Month
0,merkel juncker discuss catalan crisi eu offici,brussel reuter german chancellor angela merke...,worldnews,2017-10-07,0,2017.0,October
1,liber hack kati couric say fake news “tear ap...,kati couric wolf sheep cloth dan rather brian ...,left-news,2017-07-22,1,2017.0,July
2,trump right audit reveal state dept cybersecur...,new document appear show pattern deterior secu...,politics,2015-10-19,1,2015.0,October
3,clay aiken say ‘fg dumbass’ think trump wasn’...,former american idol finalist celebr appren...,News,2017-08-16,1,2017.0,August
4,us militia gird troubl presidenti elect near,jackson ga reuter georgia countri road camou...,politicsNews,2016-11-02,0,2016.0,November


In [43]:
# save the cleaned data
df.to_csv('../data/cleaned_data.csv', index=False)


### We can now proceed to data analysis.